In [ ]:
%load_ext autoreload
%autoreload 2

from dataloaders.loaders import generate_prompts
import config
from openai import OpenAI
from tqdm.auto import tqdm
from pathlib import Path
import json
import time
import random

In [ ]:
prompts = generate_prompts("augment_attitudes")

In [ ]:
prompts

In [ ]:
inputs = []
for prompt in prompts:
    input = [
                {"role": "user", "content": prompt},
            ]
    inputs.append(input)

In [ ]:
api_token = config.open_ai_token

In [ ]:
org = config.org

In [ ]:
client = OpenAI(organization=org, api_key=api_token)

In [ ]:
inputs

In [ ]:
responses = []
for request in tqdm(inputs):
    while True:
        num_retries = 0
        delay = 1.
        try:
            response = client.chat.completions.create(
            model="gpt-4-1106-preview",
            messages=request,
            max_tokens=1000,
            temperature=1.0,
            seed=42,
            top_p=.95
            )
            responses.append(response)
            break
        except Exception as e:
            num_retries += 1
            print(e)

            # Check if max retries has been reached
            if num_retries > 10:
                raise Exception(
                    f"Maximum number of retries (10) exceeded."
                )

            # Increment the delay
            delay *= 2 * (1 + random.random())
            print(f"Retrying with delay {delay}")

            # Sleep for the delay
            time.sleep(delay)

In [ ]:
all_preds = [[{"generated_text": response.choices[0].message.content}] for response in responses]

In [ ]:
result_path = Path("./data/att_augment.json")
result_path.parent.mkdir(exist_ok=True, parents=True)
with open(result_path, "w") as file:
    json.dump(all_preds, file)

In [ ]:
responses_json = [response.model_dump_json() for response in responses]

In [ ]:
response_path = Path("./responses/att_augment.json")
response_path.parent.mkdir(exist_ok=True, parents=True)
with open(response_path, "w") as file:
    json.dump(responses_json, file)